In [1]:
import os
import glob
import pandas as pd
from pathlib import Path

In [6]:
root_dir = "/mnt/nfs/stricon_data/VELAS_data/"
root = Path(root_dir)
subj_test_list = []
subjects = sorted([p for p in root.iterdir() if p.is_dir() and p.name.startswith("sub-")])
for subj in subjects:
    subj_test_list.append(subj)
    
subj_test_list = subj_test_list[:3]
subj_test_list

[PosixPath('/mnt/nfs/stricon_data/VELAS_data/sub-0029'),
 PosixPath('/mnt/nfs/stricon_data/VELAS_data/sub-0043'),
 PosixPath('/mnt/nfs/stricon_data/VELAS_data/sub-0057')]

In [4]:
def find_first(paths: list[Path]) -> Path | None:
    return paths[0] if len(paths) > 0 else None


def find_fmriprep_rest_files(subject_dir: Path) -> tuple[Path, Path, Path]:
    """
    Returns (bold_preproc, brain_mask, confounds_tsv) for a subject.
    Searches broadly but prefers task-rest if present.
    """
    # Prefer task-rest patterns
    bold_candidates = sorted(subject_dir.rglob("*task-rest*desc-preproc_bold.nii.gz"))
    mask_candidates = sorted(subject_dir.rglob("*task-rest*desc-brain_mask.nii.gz"))
    conf_candidates = sorted(subject_dir.rglob("*task-rest*desc-confounds_timeseries.tsv"))

    # Fallback: any desc-preproc_bold, any desc-brain_mask, any confounds
    if not bold_candidates:
        bold_candidates = sorted(subject_dir.rglob("*desc-preproc_bold.nii.gz"))
    if not mask_candidates:
        mask_candidates = sorted(subject_dir.rglob("*desc-brain_mask.nii.gz"))
    if not conf_candidates:
        conf_candidates = sorted(subject_dir.rglob("*desc-confounds_timeseries.tsv"))

    bold = find_first(bold_candidates)
    mask = find_first(mask_candidates)
    conf = find_first(conf_candidates)

    if bold is None or mask is None or conf is None:
        missing = []
        if bold is None:
            missing.append("desc-preproc_bold.nii.gz")
        if mask is None:
            missing.append("desc-brain_mask.nii.gz")
        if conf is None:
            missing.append("desc-confounds_timeseries.tsv")
        raise FileNotFoundError(f"Missing files in {subject_dir}: {', '.join(missing)}")

    return bold, mask, conf

In [12]:
out_root = "/mnt/data/CST_loops/Stricon"
for subj in subj_test_list:

    sub_dir = Path(subj)
    sub_id = sub_dir.name
    out_root = Path(out_root)
    bold, mask, conf = find_fmriprep_rest_files(sub_dir)
    out_file = out_root / sub_id / f"{sub_id}_task-rest_desc-denoised_bold.nii.gz"
    if bold.exists() and mask.exists() and conf.exists():
        #print(f'Bold file: {os.path.basename(bold)}, Mask file: {os.path.basename(mask)}, Conf File: {os.path.basename(conf)}')
        print(out_file)


/mnt/data/CST_loops/Stricon/sub-0029/sub-0029_task-rest_desc-denoised_bold.nii.gz
/mnt/data/CST_loops/Stricon/sub-0043/sub-0043_task-rest_desc-denoised_bold.nii.gz
/mnt/data/CST_loops/Stricon/sub-0057/sub-0057_task-rest_desc-denoised_bold.nii.gz
